# Goodreads Books Reviews


#### Imports

In [1]:
!pip install transformers datasets

     |████████████████████████████████| 4.0 MB 7.1 MB/s 
     |████████████████████████████████| 325 kB 74.9 MB/s 
     |████████████████████████████████| 895 kB 61.5 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 136 kB 62.8 MB/s 
     |████████████████████████████████| 212 kB 76.4 MB/s 
     |████████████████████████████████| 1.1 MB 47.7 MB/s 
     |████████████████████████████████| 127 kB 57.1 MB/s 
     |████████████████████████████████| 144 kB 55.2 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 271 kB 69.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [2]:
# Import necessary libraries

from google.colab import drive

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset
from huggingface_hub import notebook_login

import torch

#### Load Data

In [3]:
# Allow access to Google Drive files

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Unzipping data files

!unzip gdrive/My\ Drive/Colab\ Notebooks/goodreads_books_reviews/data/goodreads_books_reviews.zip

Archive:  gdrive/My Drive/Colab Notebooks/goodreads_books_reviews/data/goodreads_books_reviews.zip
  inflating: goodreads_sample_submission.csv  
  inflating: goodreads_test.csv      
  inflating: goodreads_train.csv     


In [5]:
goodreads_train = load_dataset('csv', data_files='goodreads_train.csv')['train']
goodreads_test = load_dataset('csv', data_files='goodreads_test.csv')['train']

Using custom data configuration default-865a18ef15c9438e


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-865a18ef15c9438e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-1ab66eb72bc343ae


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1ab66eb72bc343ae/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

#### Exploratory Data Analysis

In [6]:
# Easier to inspect with pandas

goodreads_train_df = goodreads_train.to_pandas()
goodreads_test_df = goodreads_test.to_pandas()

In [7]:
goodreads_train_df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,None,None,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,None,None,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [8]:
goodreads_test_df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


In [9]:
goodreads_train_df['rating'].value_counts().sort_index()

0     30988
1     28718
2     72627
3    188972
4    313688
5    265007
Name: rating, dtype: int64

#### Preprocessing

In [10]:
# Dropping all columns except review_text and rating

dropped_columns = ['user_id', 'book_id', 'review_id', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']

goodreads_train = goodreads_train.remove_columns(dropped_columns)
goodreads_test = goodreads_test.remove_columns(dropped_columns)

# Renaming text and label columns

goodreads_train = goodreads_train.rename_column('review_text', 'text')
goodreads_train = goodreads_train.rename_column('rating', 'label')
goodreads_test = goodreads_test.rename_column('review_text', 'text')

In [11]:
goodreads_train

Dataset({
    features: ['label', 'text'],
    num_rows: 900000
})

In [12]:
goodreads_test

Dataset({
    features: ['text'],
    num_rows: 478033
})

In [13]:
# Splitting training data into a training and validation set as test dataset provided is not labelled

goodreads_train_split = goodreads_train.train_test_split(test_size=0.2)

In [14]:
goodreads_train_split

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 720000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 180000
    })
})

In [15]:
# Taking only a small sample of the training data due to computational limitations

def reduce_dataset(dataset):
    total_rows = dataset.num_rows
    sample_size = int(total_rows * 0.005)
    sample_indices = np.random.RandomState(42).randint(total_rows, size=sample_size)
    return dataset.select(sample_indices)

train_data = reduce_dataset(goodreads_train_split['train'])
test_data = reduce_dataset(goodreads_train_split['test'])

In [16]:
train_data

Dataset({
    features: ['label', 'text'],
    num_rows: 3600
})

In [17]:
test_data

Dataset({
    features: ['label', 'text'],
    num_rows: 900
})

In [18]:
# Labels need to be encoded to ClassLabel type for custom datasets

train_data = train_data.class_encode_column('label')

Stringifying the column:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/4 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
train_data.features

{'label': ClassLabel(num_classes=6, names=['0', '1', '2', '3', '4', '5'], id=None),
 'text': Value(dtype='string', id=None)}

In [20]:
test_data = test_data.class_encode_column('label')

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
test_data.features

{'label': ClassLabel(num_classes=6, names=['0', '1', '2', '3', '4', '5'], id=None),
 'text': Value(dtype='string', id=None)}

In [22]:
# Tokenizing review text

tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

def tokenize_text(data):
    return tokenizer(data['text'], padding=True, truncation=True, max_length=512)
 
tokenized_train = train_data.map(tokenize_text, batched=True)
tokenized_test = test_data.map(tokenize_text, batched=True)

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [23]:
# Convert training samples to PyTorch tensors and concatenate them with the correct amount of padding to speed up training

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', max_length=512, return_tensors='pt')

In [24]:
tokenized_train

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3600
})

#### Model Building

In [25]:
model = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=6)

Downloading:   0%|          | 0.00/127M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Defining metrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
   
    #accuracy = accuracy_score(y_true=labels, y_pred=logits)
    #precision = precision_score(y_true=labels, y_pred=logits, average=None)
    #recall = recall_score(y_true=labels, y_pred=logits, average=None)
    #f1 = f1_score(y_true=labels, y_pred=logits, average='weighted')
    
    #return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

#### Model Training

In [27]:
# Training hyperparameters

training_args = TrainingArguments(
    output_dir='test_trainer',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=len(tokenized_train)//16,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

In [28]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   #compute_metrics=compute_metrics,
)

Using amp half precision backend


In [29]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3600
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1125


Epoch,Training Loss,Validation Loss
1,1.531000,1.530715
2,1.415100,1.403900
3,1.307400,1.341866
4,1.236800,1.316382
5,1.185800,1.301607


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 900
  Batch size = 16
Saving model checkpoint to test_trainer/checkpoint-225
Configuration saved in test_trainer/checkpoint-225/config.json
Model weights saved in test_trainer/checkpoint-225/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-225/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-225/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 900
  Batch siz

TrainOutput(global_step=1125, training_loss=1.3352496473524305, metrics={'train_runtime': 513.6072, 'train_samples_per_second': 35.046, 'train_steps_per_second': 2.19, 'total_flos': 1185794297856000.0, 'train_loss': 1.3352496473524305, 'epoch': 5.0})

In [30]:
# fix metrics
# find out about collator and padding/truncation